In [ ]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pickle

In [ ]:
# Load lane change data from pickle file
lane_change_dict = pd.read_pickle('data/lane_change_dict.pickle')

In [ ]:
## Utility functions to get means and stds of fields
def get_mean_and_std(groups,field_name):
    mean_list = list()
    std_list = list()
    lc_frame_index_list = list()
    for lc_frame_index, group_df in groups:
        mean_list.append(group_df[field_name].mean())
        std_list.append(group_df[field_name].std())
        lc_frame_index_list.append(lc_frame_index)
    return lc_frame_index_list, mean_list, std_list

def plot_mean_and_std(groups,field_name):
    lc_frame_index_list,mean_list, std_list = get_mean_and_std(groups,field_name)
    plt.plot(np.array(lc_frame_index_list)*0.1,mean_list)
    std_array = np.array(std_list)
    mean_array = np.array(mean_list)
    plt.fill_between(np.array(lc_frame_index_list)*0.1, mean_array - std_array, mean_array + std_array, alpha=0.25, color='b')

In [ ]:
# Prepare data for plotting
matplotlib.rcParams.update({'font.size': 14})
lane_pairs_list = [(6,5),(5,4),(4,3),(3,2),(2,1)]
lane_change_range_all_list = []
for lane_pair in lane_pairs_list:
    lane_change_range_all_list.extend(lane_change_dict[lane_pair])
    
lc_range_all_df = pd.concat(lane_change_range_all_list)
lc_range_all_df.vel_x = lc_range_all_df.vel_x*(-1.0)
total_lc =  len(lane_change_range_all_list)
lane_pairs_list_temp = lane_pairs_list

lane_pairs_list = [(1,2),(2,3),(3,4),(4,5),(5,6)]
lane_change_range_all_list = []
for lane_pair in lane_pairs_list:
    lane_change_range_all_list.extend(lane_change_dict[lane_pair])
total_lc = total_lc + len(lane_change_range_all_list)
lc_range_all_df_temp = pd.concat(lane_change_range_all_list)
lc_range_all_df = pd.concat([lc_range_all_df,lc_range_all_df_temp])

groups = lc_range_all_df.groupby('lc_frame_index')
lane_pairs_list.extend(lane_pairs_list_temp)
print lane_pairs_list

In [ ]:
# Generate lateral pos figures
fig = plt.figure(figsize=(15,5))
lc_count = 0
for lane_pair in lane_pairs_list:
    lc_df_list = lane_change_dict[lane_pair]
    for ll in lc_df_list:
#         plt.subplot(2,1,1)
        plt.plot(np.array(ll.lc_frame_index)*0.1,ll.local_x_normalized,alpha=0.1)
        lc_count = lc_count + 1
print "Number of lane changes: %s" %lc_count
plt.ylabel("Normalized Lateral Position")
plt.xlabel("Relative Time to Lane-change (sec)")
# ylim = plt.gca().get_ylim()
plt.ylim([-0.5,1.5])
# plt.savefig('lanechange_pos_all.pdf')

fig = plt.figure(figsize=(15,5))
# plt.subplot(2,1,2)
plot_mean_and_std(groups,'local_x_normalized')
plt.ylim([-0.5,1.5])
plt.ylabel("Normalized Lateral Position")
plt.xlabel("Relative Time to Lane-change (sec)")
# plt.savefig('lanechange_pos_avg.pdf')

In [ ]:
# Generate lateral vel figures
fig = plt.figure(figsize=(15,5))
lc_count = 0
for lane_pair in lane_pairs_list:
    lc_df_list = lane_change_dict[lane_pair]
    for ll in lc_df_list:
        if lane_pair[0] > lane_pair[1]:
            plt.plot(np.array(ll.lc_frame_index)*0.1,-1.0*ll.vel_x,alpha=0.1)
        else:
            plt.plot(np.array(ll.lc_frame_index)*0.1,ll.vel_x,alpha=0.1)
        lc_count = lc_count + 1
print "Number of lane changes: %s" %lc_count
plt.ylabel("Lateral Velocity (Feet/sec)")
plt.xlabel("Relative Time to Lane-change (sec)")
# ylim = plt.gca().get_ylim()
plt.ylim([-10,60])
plt.savefig('lanechange_vel_x_all.pdf')

fig = plt.figure(figsize=(15,5))
# plt.subplot(2,1,2)
plot_mean_and_std(groups,'vel_x')
plt.ylim([-10,60])
plt.ylabel("Lateral Velocity (Feet/sec)")
plt.xlabel("Relative Time to Lane-change (sec)")
plt.savefig('lanechange_vel_x_avg.pdf')

In [ ]:
# Generate longditudinal velocity
fig = plt.figure(figsize=(15,5))
lc_count = 0
for lane_pair in lane_pairs_list:
    lc_df_list = lane_change_dict[lane_pair]
    for ll in lc_df_list:
        vel_start = ll.veh_vel.head(1).squeeze()
        vel_max = max(ll.veh_vel.tolist())
        plt.plot(np.array(ll.lc_frame_index)*0.1,(ll.veh_vel-vel_start)/vel_max,alpha=.1)
        lc_count = lc_count + 1
print "Number of lane changes: %s" %lc_count
plt.ylabel("Normalized Variation \n in Longditudinal Velocity")
plt.xlabel("Relative Time to Lane-change (sec)")
ylim = plt.gca().get_ylim()
# plt.ylim([-10,60])
plt.savefig('lanechange_vel_y_all.pdf')

fig = plt.figure(figsize=(15,5))
# plt.subplot(2,1,2)
plot_mean_and_std(groups,'veh_vel_normalized')
plt.ylabel("Normalized Variation \n in Longditudinal Velocity")
plt.xlabel("Relative Time to Lane-change (sec)")
plt.ylim(ylim)
# plt.savefig('lanechange_vel_y_avg.pdf')